In [1]:
from torch.utils.data import Dataset
import pandas as pd

from transformers import GPT2LMHeadModel, GPT2Tokenizer
from torch.optim import Adam
from torch.utils.data import DataLoader
# from tqdm import tqdm_notebook as tqdm
from tqdm.notebook import tqdm
import torch

import warnings
warnings.filterwarnings("ignore")

In [2]:
class ChatData(Dataset):
    def __init__(self, path:str, tokenizer):
        self.data = pd.read_csv('sample_data.csv', encoding='unicode_escape')

        self.X = []
#         for i in self.data:
#             for j in i['dialog']:
#                 self.X.append(j['text'])
                
        for idx, row in self.data.iterrows():
            self.X.append('<startofstring> ' + row['question'] + ' <bot>: ' + row['answer'] + ' <endofstring>')
#         for idx, i in enumerate(self.X):
#             try:
#                 self.X[idx] = "<startofstring> "+i+" <bot>: "+self.X[idx+1]+" <endofstring>"
#             except:
#                 break

#         self.X = self.X[:5000]
        
        print(self.X[0])

        self.X_encoded = tokenizer(self.X,max_length=40, truncation=True, padding="max_length", return_tensors="pt")
        self.input_ids = self.X_encoded['input_ids']
        self.attention_mask = self.X_encoded['attention_mask']

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return (self.input_ids[idx], self.attention_mask[idx])

In [3]:
def infer(inp):
    inp = "<startofstring> "+inp+" <bot>: "
    inp = tokenizer(inp, return_tensors="pt")
    X = inp["input_ids"].to(device)
    a = inp["attention_mask"].to(device)
    
#     model = GPT2LMHeadModel.from_pretrained('./model_state.pt', local_files_only=True)
#     model = GPT2LMHeadModel.from_pretrained("gpt2")
#     model.load_state_dict(torch.load('./model_state.pt'))
#     model.resize_token_embeddings(len(tokenizer))
    
    output = model.generate(X, attention_mask=a )
    output = tokenizer.decode(output[0])
    return output

In [4]:
def train(chatData, model, optim):

    epochs = 100

    for i in tqdm(range(epochs)):
        for X, a in chatData:
            X = X.to(device)
            a = a.to(device)
            optim.zero_grad()
            loss = model(X, attention_mask=a, labels=X).loss
            loss.backward()
            optim.step()
        torch.save(model.state_dict(), "model_state.pt")
        print(infer("Do I need a visa to visit Canada?"))

In [5]:
device = "cuda" if torch.cuda.is_available() else "mps" if torch.backends.mps.is_available() else "cpu"

tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
tokenizer.add_special_tokens({"pad_token": "<pad>", 
                                "bos_token": "<startofstring>",
                                "eos_token": "<endofstring>"})
tokenizer.add_tokens(["<bot>:"])

1

In [6]:
model = GPT2LMHeadModel.from_pretrained("gpt2")
model.resize_token_embeddings(len(tokenizer))

model = model.to(device)

You are resizing the embedding layer without providing a `pad_to_multiple_of` parameter. This means that the new embeding dimension will be 50261. This might induce some performance reduction as *Tensor Cores* will not be available. For more details  about this, or help on choosing the correct value for resizing, refer to this guide: https://docs.nvidia.com/deeplearning/performance/dl-performance-matrix-multiplication/index.html#requirements-tc


In [7]:
# print(tokenizer.decode(model.generate(**tokenizer("hey i was good at basketball but ",
#                          return_tensors="pt"))[0]))

In [8]:
chatData = ChatData("./chat_data.json", tokenizer)
chatData =  DataLoader(chatData, batch_size=64)

<startofstring> Do I need a visa to visit Canada?   <bot>: Most travelers need a visa or an Electronic Travel authorization (eTA) to fly to or transit through a Canadian airport.  <endofstring>


In [9]:
model.train()

optim = Adam(model.parameters(), lr=1e-3)

print("training .... ")
train(chatData, model, optim)

training .... 


  0%|          | 0/100 [00:00<?, ?it/s]

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


<startofstring> Do I need a visa to visit Canada? <bot>: TheTheTheTheTheTheTheTheThe


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


...

<startofstring> Do I need a visa to visit Canada? <bot>: Most travelers need a visa or an Electronic Travel


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


<startofstring> Do I need a visa to visit Canada? <bot>: Most travelers need a visa or an Electronic Travel


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


<startofstring> Do I need a visa to visit Canada? <bot>: Most travelers need a visa or an Electronic Travel


In [10]:
print("infer from model : ")
while True:
    inp = input()
    if inp == 'exit':
        break
    print(infer(inp))

infer from model : 
Do I need a visa to visit Canada?


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


<startofstring> Do I need a visa to visit Canada? <bot>: Most travelers need a visa or an Electronic Travel
What is a business visitor?


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


<startofstring> What is a business visitor? <bot>: A business visitor is someone who comes to Canada for international business
Who has to take the citizenship test?


...

<startofstring> Can I sponsor a parent or grandparent? <bot>: Yes, as a Canadian citizen or permanent resident
What is the fee for an Access request?


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


<startofstring> What is the fee for an Access request? <bot>: Each request filed under the Access to Information Act
What is the fee for a Privacy request?


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


<startofstring> What is the fee for a Privacy request? <bot>: There are no fees under the Privacy Act.
exit
